# 청담대교_유량
curve fit으로 결측치가 (눈으로 보기에) 제대로 처리되었는지 확인

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('/content/drive/MyDrive/신입기수 프로젝트/Dataset')

### rawdata

In [3]:
raw_data = pd.read_csv('rawdata.csv')
raw_data

,날짜,현재수위,유입량,저수량,공용량,방류량,강화대교_조위,청담대교_유량,잠수교_유량,한강대교_유량,행주대교_유량,대곡교_강수량,진관교_강수량,송정동_강수량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위
0,2012-05-01 00:00,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0,310.7,300.2,290.0,275.3
1,2012-05-01 00:10,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0,314.7,300.2,290.0,275.3
2,2012-05-01 00:20,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0,313.7,301.2,290.0,275.3
3,2012-05-01 00:30,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0,311.7,301.2,290.0,276.3
4,2012-05-01 00:40,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10,25.040,259.23,212.86,31.14,259.23,510.0,319.84,NaN,-456.41,974.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276332,2022-07-18 23:20,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276333,2022-07-18 23:30,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276334,2022-07-18 23:40,25.040,259.13,212.86,31.14,259.13,458.0,454.91,NaN,-1023.37,1073.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# 데이터셋 
raw_data = raw_data[['날짜', '방류량']]
raw_data.날짜 = pd.to_datetime(raw_data.날짜)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Curve fit한 데이터셋

In [5]:
curvefit = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/0822_팔당댐_Arima.csv', encoding = 'cp949')
curvefit

,날짜,공용량,현재수위,방류량
0,2012-05-01 00:00,24.93,24.800,555.00
1,2012-05-01 00:10,25.15,24.794,562.90
2,2012-05-01 00:20,25.31,24.789,576.40
3,2012-05-01 00:30,25.31,24.789,563.10
4,2012-05-01 00:40,25.31,24.789,576.40
...,...,...,...,...
276331,2022-07-18 23:10,31.14,25.040,259.23
276332,2022-07-18 23:20,31.14,25.040,260.46
276333,2022-07-18 23:30,31.14,25.040,259.37
276334,2022-07-18 23:40,31.14,25.040,259.13


In [6]:
curvefit = curvefit[['날짜', '방류량']]

In [7]:
# 데이터셋 
curvefit.날짜 = pd.to_datetime(curvefit.날짜)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### 그래프 그릴 인덱스

In [8]:
# 채워야 하는 인덱스
nan_index = list(raw_data[raw_data['방류량'].isnull()].index)
nan_index

[13200,
 68081,
 68083,
 68084,
 68085,
 68086,
 68087,
 68089,
 68090,
 68091,
 68092,
 68093,
 68095,
 68096,
 68097,
 68098,
 68099,
 68101,
 68102,
 68103,
 68104,
 81560,
 81561,
 81562,
 81563,
 81565,
 81566,
 81567,
 81568,
 82337,
 84037,
 84319,
 84597,
 100174,
 123622,
 123623,
 123625,
 123626,
 123627,
 123628,
 123629,
 123631,
 132986,
 132987,
 132988,
 132989,
 132991,
 132992,
 132993,
 132994,
 132995,
 132997,
 132998,
 132999,
 133003,
 133004,
 139637,
 154512,
 156158,
 156159,
 157815,
 157911,
 158739,
 159615,
 159633,
 159747,
 160833,
 161121,
 161157,
 161889,
 162009,
 162027,
 162135,
 162459,
 162555,
 162567,
 162627,
 162639,
 162645,
 162699,
 162711,
 162837,
 162909,
 163011,
 163107,
 163191,
 163323,
 163335,
 163359,
 163365,
 163467,
 163515,
 163617,
 163845,
 164019,
 164511,
 164763,
 165123,
 165147,
 165183,
 165428,
 165429,
 165430,
 165431,
 165432,
 165433,
 165434,
 165435,
 165436,
 165437,
 165438,
 165439,
 165440,
 165441,
 165442

In [9]:
len(nan_index)

743

In [10]:
unconsecutive = []
consecutive = []

i = 0
while i < len(nan_index):
    if nan_index[i]+1 in nan_index: # 다음 거가 nanIndex 안에 있으면 다음 거까지 consecutive에 추가
        consecutive.extend([nan_index[i], nan_index[i+1]])
        try:
            i += 2
        except: #i가 len(nanIndex2) - 1
            break
    elif nan_index[i]-1 in nan_index: #다음 거는 nanIndex 안에 없지만 consecutive로 들어가야 하는 경우(반복 구간의 마지막 원소)
        consecutive.append(nan_index[i])
        i += 1
    else:
        unconsecutive.append(nan_index[i])
        i += 1

In [11]:
# consecutive 내에서 구간별로 분리
dummy = []
consecutive2 = [] # 2차원 consecutive
for i in consecutive: 
    dummy.append(i)
    if i+1 not in consecutive:
        if i == consecutive[-2]: #만약 마지막 반복일 경우, consecutive[-1]을 추가로 넣어주어야 함
            dummy.append(i+1)
            consecutive2.append(dummy)
        else: 
            consecutive2.append(dummy)
            dummy= []   

In [12]:
print(len(consecutive)+ len(unconsecutive) == len(nan_index))
print(len(sum(consecutive2, [])) == len(consecutive))

True
True


In [13]:
len(consecutive2)

38

### 시각화

In [14]:
# 왼: rawdata 분포, 오: Curve fit 처리
# (55, 2, n)
# f, axes = plt.subplots(len(consecutive2), 1)
# f.set_size_inches((20, 65))
# plt.subplots_adjust(wspace = 0.3, hspace = 0.3)

fig = plt.figure()
fig.set_size_inches((20, 650))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
plt.title('Left: Raw data, Right: Curve fit', fontsize = 20)

n = 0
for i in range(1, len(consecutive2)*2+1, 2):
    start = consecutive2[n][0]-50
    end = consecutive2[n][-1]+50
    
    ax1 = fig.add_subplot(586, 2, i)
    x1 = range(len(raw_data['날짜'][start:end]))
    y1 = raw_data['방류량'][start:end]
    ax1.plot(x1, y1, color = 'green')

    ax2 = fig.add_subplot(586, 2, i+1)
    x2 = range(len(raw_data['날짜'][start:end]))
    y2 = curvefit['방류량'][start:end]
    ax2.plot(x2, y2, color = 'skyblue')
    n += 1

plt.tight_layout()
plt.show()